# Process terms 
Process terms retrieved from the neural network model using `LIME`

---
## Load the retrieved terms

In [1]:
import os

In [2]:
version = 'v2'

In [3]:
root_dir = '../..'
data_dir = 'data/terms'
filename = f'relevant_terms_wikidata_nn_{version}.csv'
filepath = os.path.join(root_dir, data_dir, filename)

In [4]:
import pandas as pd

In [5]:
terms_df = pd.read_csv(filepath)

In [6]:
terms_df.head()

,label,term,weight,data_id
0,Q1754,stockholm,0.555859,0
1,Q1754,sweden,0.406443,0
2,Q1754,city,0.138746,0
3,Q1754,capital,0.130178,0
4,Q1754,stockholm,0.395502,1


---

## Rank terms

### Add `rank` column
Add a ranking column by sorting each document's terms by their LIME weight in descending order

In [7]:
terms_df['rank'] = terms_df.groupby(['label', 'data_id'])['weight'].rank(ascending=False)

In [8]:
terms_df.head()

,label,term,weight,data_id,rank
0,Q1754,stockholm,0.555859,0,1.0
1,Q1754,sweden,0.406443,0,2.0
2,Q1754,city,0.138746,0,3.0
3,Q1754,capital,0.130178,0,4.0
4,Q1754,stockholm,0.395502,1,1.0


### Compute stats

In [9]:
terms_stats_df_1 = terms_df.groupby(['label', 'term']).agg({'weight': 'sum','rank': 'mean'})
terms_stats_df_2 = terms_df.groupby(['label', 'term']).size().to_frame('count')
terms_stats_df = terms_stats_df_1.join(terms_stats_df_2).reset_index()

In [10]:
terms_stats_df.head()

,label,term,weight,rank,count
0,Q1005682,2010_census,0.304872,5.500000,2
1,Q1005682,513,0.418916,3.000000,2
2,Q1005682,carver,0.563979,5.400000,5
3,Q1005682,city,0.409819,6.000000,4
4,Q1005682,county,1.060182,5.333333,9


### Ranking 
Rank terms by mean ranking (ascending) and total weight (descending)

In [11]:
ranking1_df = terms_stats_df.groupby('label')\
                .apply(lambda grp: grp.sort_values(by=['rank', 'weight'], ascending=[True, False]))\
                .reset_index(drop=True)

In [12]:
ranking1_df.groupby('label').head()

,label,term,weight,rank,count
0,Q1005682,hamburg,6.433465,1.000000,11
1,Q1005682,states,2.278421,2.454545,11
2,Q1005682,513,0.418916,3.000000,2
3,Q1005682,minnesota,1.838184,3.100000,10
4,Q1005682,united,2.012429,3.636364,11
...,...,...,...,...,...
3818,Q998718,hamburg,2.675068,1.222222,9
3819,Q998718,iowa,1.819555,1.600000,5
3820,Q998718,nishnabotna,1.040748,2.600000,5
3821,Q998718,fremont,0.865026,3.250000,4


---

## Save selected ranking

In [13]:
ranking_filename = f'ranking_wikidata_nn_{version}.xlsx'
ranking_filepath = os.path.join(root_dir, data_dir, ranking_filename)

In [14]:
ranking_filepath

'../../data/terms/ranking_wikidata_nn_v2.xlsx'

In [15]:
ranking1_df.to_excel(ranking_filepath)

---